# Parte 2: Data Science

## **Sumário**

* 1. Lendo Base de Dados
* 2. Algoritmo Apriori
    * 2.1. Calculo Support(P(A))
    * 2.2. Calculo Support(P(A, B))
    * 2.3. Calculo Support(P(A,B,C))
    * 2.4. Join Resultados

## 1. Lendo Base de Dados

In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

df = pd.read_csv('/home/rafaelfabrichimidt/Documentos/Projetos/Python/codigos/case/analytics_engineer_carrefour/tutorial/CASE_PRATICO_SAMS_CLUB.csv')

df['periodo'] = pd.to_datetime(df['periodo'])
df['cesta'] = df['socio'].astype('str') + '-' + df['ticket'].astype('str')

indices = df[df['item_unidade'] <= 0].index

df.drop(labels = indices, axis = 0, inplace = True)

print('Qtd de linhas = ', df.shape[0])
print('Qtd de colunas = ', df.shape[1])

Qtd de linhas =  874860
Qtd de colunas =  12


In [3]:
df.head()

,periodo,id_clube,socio,canal,ticket,departamento,item_id,item_descricao,item_unidade,receita_bruta,margem,cesta
0,2022-01-10,5235,45765100809381908,PISO,4576510080938190820220110,BAZAR E TÊXTIL,1259475,CALCAO,2,81.17,22.43,45765100809381908-4576510080938190820220110
1,2022-01-10,5235,45766100903265726,PISO,4576610090326572620220110,BAZAR E TÊXTIL,1259475,CALCAO,1,41.27,11.90,45766100903265726-4576610090326572620220110
2,2022-01-10,5235,45765100815078019,PISO,4576510081507801920220110,BAZAR E TÊXTIL,1225017,CHAPEU PRAIA,1,51.27,11.10,45765100815078019-4576510081507801920220110
3,2022-01-10,5235,45766100904345030,PISO,4576610090434503020220110,BAZAR E TÊXTIL,1216504,BANCO DOBRAVEL,1,81.34,22.93,45766100904345030-4576610090434503020220110
4,2022-01-10,5235,45766100903286284,PISO,4576610090328628420220110,BAZAR E TÊXTIL,1259473,BERMUDA MOLETOM,2,201.17,46.37,45766100903286284-4576610090328628420220110


## 2. Algoritmo Apriori


### 2.1. Calculo Support(P(A))

In [4]:
support_A = df.groupby(['item_descricao'])['item_unidade'].sum().sort_values(ascending = False)[0:10] / df.shape[0]
dic_support_A = dict(support_A)
dic_support_A

{'SACOLA PLASTICA MM BRANCA': 0.03334819285371374,
 'SACOLA PLASTICA MM CINZA': 0.009388930800356628,
 'PAPEL TOALHA KITCHEN JUMB FD C360FL': 0.008732825823560341,
 'SACOLA PLASTICA MM VERDE': 0.008645954781336443,
 'CR DE LEITE PIRACANJUBA TP 200G': 0.008372768214342866,
 'OLEO SOJA LIZA PET 900ML': 0.007901835722286995,
 'CERV QUILMES CLASSICA LN 340ML': 0.00673364881238141,
 'PAPEL REPORT MULT A4 BRANCO 500 F': 0.006707358891708388,
 'ACUC REF UNIAO 1KG': 0.006468463525592666,
 'MM LAVA ROUPAS 5LTS ULTIMATE CLEAN': 0.0060272500742976015}

In [15]:
df_support_A = pd.DataFrame.from_dict(dic_support_A, orient = 'index')
df_support_A.reset_index(inplace = True)
df_support_A.columns = ['A', 'value']

In [5]:
_10_items_mais_relevantes = support_A.keys().values.tolist()

### 2.2. Calculo Support(P(A, B))

In [6]:
qtd_total = df.shape[0]
dic_support_A_B = {}
limite = 0.0002

for item in _10_items_mais_relevantes:
    
    print(item)

    #filtrar as cestas que possuem esse item 
    cestas = df[df['item_descricao'] == item]['cesta'].values.tolist()
    
    #filtrar apenas essas cestas
    df_cestas = df[df['cesta'].isin(cestas)].copy()

    #contar os items que mais aparecem nessa cesta (pegar os 15 primeiros) 
    qtd_para_confidence = df_cestas['item_descricao'].value_counts()

    #qtd de cestas que o item (X) aparece
    #qtd_cestas = df_cestas.shape[0]

    dic_uniao = dict(qtd_para_confidence / qtd_total)

    dic_uniao_acima_limite = {k : v for k, v in dic_uniao.items() if v > limite}

    dic_support_A_B[item] = dic_uniao_acima_limite




SACOLA PLASTICA MM BRANCA
SACOLA PLASTICA MM CINZA
PAPEL TOALHA KITCHEN JUMB FD C360FL
SACOLA PLASTICA MM VERDE
CR DE LEITE PIRACANJUBA TP 200G
OLEO SOJA LIZA PET 900ML
CERV QUILMES CLASSICA LN 340ML
PAPEL REPORT MULT A4 BRANCO 500 F
ACUC REF UNIAO 1KG
MM LAVA ROUPAS 5LTS ULTIMATE CLEAN


In [7]:
df_support_A_B = pd.DataFrame.from_dict(dic_support_A_B, orient = 'index')
colunas = df_support_A_B.columns
df_support_A_B.reset_index(inplace = True)
df_support_A_B.rename(columns = {'index': 'B'}, inplace = True)
df_support_A_B = pd.melt(df_support_A_B, id_vars = 'B', value_vars = colunas)
df_support_A_B.rename(columns = {'variable':'A'}, inplace = True)
df_support_A_B.dropna(inplace = True)
indices = df_support_A_B[df_support_A_B['A'] == df_support_A_B['B']].index
df_support_A_B.drop(labels = indices, axis = 0, inplace  = True)
df_support_A_B.reset_index(drop = True, inplace = True)

In [8]:
df_support_A_B

,B,A,value
0,PAPEL TOALHA KITCHEN JUMB FD C360FL,SACOLA PLASTICA MM BRANCA,0.001447
1,CR DE LEITE PIRACANJUBA TP 200G,SACOLA PLASTICA MM BRANCA,0.000430
2,OLEO SOJA LIZA PET 900ML,SACOLA PLASTICA MM BRANCA,0.000287
3,PAPEL REPORT MULT A4 BRANCO 500 F,SACOLA PLASTICA MM BRANCA,0.000370
4,ACUC REF UNIAO 1KG,SACOLA PLASTICA MM BRANCA,0.000274
...,...,...,...
290,PAPEL TOALHA KITCHEN JUMB FD C360FL,VANISH 3 LT COLORS TIRA MANCHAS,0.000217
291,PAPEL TOALHA KITCHEN JUMB FD C360FL,LYSOFORM BRUTO DESINFETANTE 5LT,0.000211
292,PAPEL TOALHA KITCHEN JUMB FD C360FL,ACUC DEME ORGANICO NATIVE 1KG,0.000203
293,PAPEL TOALHA KITCHEN JUMB FD C360FL,SAPOLIO RADIUM 2X450ML,0.000201


### 2.3. Calculo Support(P(A,B,C))

In [9]:
dic_support_A_B_C = {}

for i in range(0, df_support_A_B.shape[0], 1):
    item_A = df_support_A_B.loc[i, 'A']
    item_B = df_support_A_B.loc[i, 'B']
    print('item A = {}'.format(item_A))
    print('item B = {}'.format(item_B))

    #filtrar as cestas que possuem esse item 
    cestas = df[(df['item_descricao'] == item_A)]['cesta'].values.tolist()
    
    #filtrar apenas essas cestas
    df_cestas = df[df['cesta'].isin(cestas)].copy()

    cestas = df_cestas[(df_cestas['item_descricao'] == item_B)]['cesta'].values.tolist()

    df_cestas = df[df['cesta'].isin(cestas)].copy()

    #contar os items que mais aparecem nessa cesta (pegar os 15 primeiros) 
    qtd_para_confidence = df_cestas['item_descricao'].value_counts()
        
    dic_uniao = dict(qtd_para_confidence / qtd_total)

    dic_uniao.pop(item_A)
    dic_uniao.pop(item_B)
        
    dic_uniao_acima_limite = {k : v for k, v in dic_uniao.items() if v > limite}

    dic_support_A_B_C[item_A + '-' + item_B] = dic_uniao_acima_limite

    

item A = SACOLA PLASTICA MM BRANCA
item B = PAPEL TOALHA KITCHEN JUMB FD C360FL
item A = SACOLA PLASTICA MM BRANCA
item B = CR DE LEITE PIRACANJUBA TP 200G
item A = SACOLA PLASTICA MM BRANCA
item B = OLEO SOJA LIZA PET 900ML
item A = SACOLA PLASTICA MM BRANCA
item B = PAPEL REPORT MULT A4 BRANCO 500 F
item A = SACOLA PLASTICA MM BRANCA
item B = ACUC REF UNIAO 1KG
item A = SACOLA PLASTICA MM BRANCA
item B = MM LAVA ROUPAS 5LTS ULTIMATE CLEAN
item A = PAPEL TOALHA KITCHEN JUMB FD C360FL
item B = SACOLA PLASTICA MM BRANCA
item A = PAPEL TOALHA KITCHEN JUMB FD C360FL
item B = CR DE LEITE PIRACANJUBA TP 200G
item A = PAPEL TOALHA KITCHEN JUMB FD C360FL
item B = OLEO SOJA LIZA PET 900ML
item A = PAPEL TOALHA KITCHEN JUMB FD C360FL
item B = PAPEL REPORT MULT A4 BRANCO 500 F
item A = PAPEL TOALHA KITCHEN JUMB FD C360FL
item B = ACUC REF UNIAO 1KG
item A = PAPEL TOALHA KITCHEN JUMB FD C360FL
item B = MM LAVA ROUPAS 5LTS ULTIMATE CLEAN
item A = PAPEL TOALHA KITCHEN JUMB FD C360FL
item B = SACOLA

In [18]:
df_support_A_B_C = pd.DataFrame.from_dict(dic_support_A_B_C, orient = 'index')
colunas = df_support_A_B_C.columns

df_support_A_B_C.reset_index(inplace = True)
df_support_A_B_C.rename(columns = {'index': 'A-B'}, inplace = True)
df_support_A_B_C['A'] = df_support_A_B_C['A-B'].str.split('-', expand = True)[0]
df_support_A_B_C['B'] = df_support_A_B_C['A-B'].str.split('-', expand = True)[1]
df_support_A_B_C.drop(labels = 'A-B', axis = 1, inplace = True)

df_support_A_B_C = pd.melt(df_support_A_B_C, id_vars = ['A', 'B'], value_vars = colunas)
df_support_A_B_C.rename(columns = {'variable':'C'}, inplace = True)
df_support_A_B_C.dropna(inplace = True)

df_support_A_B_C.reset_index(drop = True, inplace = True)

In [19]:
df_support_A_B_C

,A,B,C,value
0,SACOLA PLASTICA MM BRANCA,PAPEL TOALHA KITCHEN JUMB FD C360FL,PH NEVE FT 20M L32P28UN,0.000226
1,PAPEL TOALHA KITCHEN JUMB FD C360FL,SACOLA PLASTICA MM BRANCA,PH NEVE FT 20M L32P28UN,0.000226
2,PH NEVE FT 20M L32P28UN,SACOLA PLASTICA MM BRANCA,PAPEL TOALHA KITCHEN JUMB FD C360FL,0.000224
3,PH NEVE FT 20M L32P28UN,PAPEL TOALHA KITCHEN JUMB FD C360FL,SACOLA PLASTICA MM BRANCA,0.000249
4,PH PERSONAL VIP FD 40 ROLOSX30M,PAPEL TOALHA KITCHEN JUMB FD C360FL,SACOLA PLASTICA MM BRANCA,0.000222


### 2.4. Join Resultados

In [16]:
df_support_A.head(3)

,A,value
0,SACOLA PLASTICA MM BRANCA,0.033348
1,SACOLA PLASTICA MM CINZA,0.009389
2,PAPEL TOALHA KITCHEN JUMB FD C360FL,0.008733


In [17]:
df_support_A_B.head(3)

,B,A,value
0,PAPEL TOALHA KITCHEN JUMB FD C360FL,SACOLA PLASTICA MM BRANCA,0.001447
1,CR DE LEITE PIRACANJUBA TP 200G,SACOLA PLASTICA MM BRANCA,0.000430
2,OLEO SOJA LIZA PET 900ML,SACOLA PLASTICA MM BRANCA,0.000287


In [20]:
df_support_A_B_C.head(3)

,A,B,C,value
0,SACOLA PLASTICA MM BRANCA,PAPEL TOALHA KITCHEN JUMB FD C360FL,PH NEVE FT 20M L32P28UN,0.000226
1,PAPEL TOALHA KITCHEN JUMB FD C360FL,SACOLA PLASTICA MM BRANCA,PH NEVE FT 20M L32P28UN,0.000226
2,PH NEVE FT 20M L32P28UN,SACOLA PLASTICA MM BRANCA,PAPEL TOALHA KITCHEN JUMB FD C360FL,0.000224


In [68]:
df_support_A['status'] = 1
df_support_A_B['status'] = 2
df_support_A_B_C['status'] = 3

df_final = pd.concat([df_support_A, df_support_A_B, df_support_A_B_C])

df_final = df_final[['A', 'B', 'C' ,'value', 'status']]

df_final.reset_index(inplace = True, drop = True)

df_final.fillna('', inplace = True)

In [69]:
df_final

,A,B,C,value,status
0,SACOLA PLASTICA MM BRANCA,,,0.033348,1
1,SACOLA PLASTICA MM CINZA,,,0.009389,1
2,PAPEL TOALHA KITCHEN JUMB FD C360FL,,,0.008733,1
3,SACOLA PLASTICA MM VERDE,,,0.008646,1
4,CR DE LEITE PIRACANJUBA TP 200G,,,0.008373,1
...,...,...,...,...,...
305,SACOLA PLASTICA MM BRANCA,PAPEL TOALHA KITCHEN JUMB FD C360FL,PH NEVE FT 20M L32P28UN,0.000226,3
306,PAPEL TOALHA KITCHEN JUMB FD C360FL,SACOLA PLASTICA MM BRANCA,PH NEVE FT 20M L32P28UN,0.000226,3
307,PH NEVE FT 20M L32P28UN,SACOLA PLASTICA MM BRANCA,PAPEL TOALHA KITCHEN JUMB FD C360FL,0.000224,3
308,PH NEVE FT 20M L32P28UN,PAPEL TOALHA KITCHEN JUMB FD C360FL,SACOLA PLASTICA MM BRANCA,0.000249,3


In [73]:
df_final.sort_values(by = 'value', ascending = False)[0:15]

,A,B,C,value,status
0,SACOLA PLASTICA MM BRANCA,,,0.033348,1
1,SACOLA PLASTICA MM CINZA,,,0.009389,1
2,PAPEL TOALHA KITCHEN JUMB FD C360FL,,,0.008733,1
3,SACOLA PLASTICA MM VERDE,,,0.008646,1
4,CR DE LEITE PIRACANJUBA TP 200G,,,0.008373,1
5,OLEO SOJA LIZA PET 900ML,,,0.007902,1
6,CERV QUILMES CLASSICA LN 340ML,,,0.006734,1
7,PAPEL REPORT MULT A4 BRANCO 500 F,,,0.006707,1
8,ACUC REF UNIAO 1KG,,,0.006468,1
9,MM LAVA ROUPAS 5LTS ULTIMATE CLEAN,,,0.006027,1


In [74]:
df_final[df_final['status'] == 3]

,A,B,C,value,status
305,SACOLA PLASTICA MM BRANCA,PAPEL TOALHA KITCHEN JUMB FD C360FL,PH NEVE FT 20M L32P28UN,0.000226,3
306,PAPEL TOALHA KITCHEN JUMB FD C360FL,SACOLA PLASTICA MM BRANCA,PH NEVE FT 20M L32P28UN,0.000226,3
307,PH NEVE FT 20M L32P28UN,SACOLA PLASTICA MM BRANCA,PAPEL TOALHA KITCHEN JUMB FD C360FL,0.000224,3
308,PH NEVE FT 20M L32P28UN,PAPEL TOALHA KITCHEN JUMB FD C360FL,SACOLA PLASTICA MM BRANCA,0.000249,3
309,PH PERSONAL VIP FD 40 ROLOSX30M,PAPEL TOALHA KITCHEN JUMB FD C360FL,SACOLA PLASTICA MM BRANCA,0.000222,3


In [63]:
indices = df_final[df_final['status'] == 1].index
df_final.loc[indices, 'P'] = '(' + df_final.loc[indices,'A'] + ')'

indices = df_final[df_final['status'] == 2].index
df_final.loc[indices, 'P'] = '(' + df_final.loc[indices,'A'] + ' - ' + df_final.loc[indices,'B'] + ')'

indices = df_final[df_final['status'] == 3].index
df_final.loc[indices, 'P'] = '(' + df_final.loc[indices,'A'] + ' - ' + df_final.loc[indices,'B'] + ' - ' + df_final.loc[indices,'C'] + ')'

In [65]:
df_final.fillna('', inplace = True)
df_final

,A,B,C,value,status,P
0,SACOLA PLASTICA MM BRANCA,,,0.033348,1,(SACOLA PLASTICA MM BRANCA)
1,SACOLA PLASTICA MM CINZA,,,0.009389,1,(SACOLA PLASTICA MM CINZA)
2,PAPEL TOALHA KITCHEN JUMB FD C360FL,,,0.008733,1,(PAPEL TOALHA KITCHEN JUMB FD C360FL)
3,SACOLA PLASTICA MM VERDE,,,0.008646,1,(SACOLA PLASTICA MM VERDE)
4,CR DE LEITE PIRACANJUBA TP 200G,,,0.008373,1,(CR DE LEITE PIRACANJUBA TP 200G)
...,...,...,...,...,...,...
305,SACOLA PLASTICA MM BRANCA,PAPEL TOALHA KITCHEN JUMB FD C360FL,PH NEVE FT 20M L32P28UN,0.000226,3,(SACOLA PLASTICA MM BRANCA - PAPEL TOALHA KITC...
306,PAPEL TOALHA KITCHEN JUMB FD C360FL,SACOLA PLASTICA MM BRANCA,PH NEVE FT 20M L32P28UN,0.000226,3,(PAPEL TOALHA KITCHEN JUMB FD C360FL - SACOLA ...
307,PH NEVE FT 20M L32P28UN,SACOLA PLASTICA MM BRANCA,PAPEL TOALHA KITCHEN JUMB FD C360FL,0.000224,3,(PH NEVE FT 20M L32P28UN - SACOLA PLASTICA MM ...
308,PH NEVE FT 20M L32P28UN,PAPEL TOALHA KITCHEN JUMB FD C360FL,SACOLA PLASTICA MM BRANCA,0.000249,3,(PH NEVE FT 20M L32P28UN - PAPEL TOALHA KITCHE...
